In [1]:
%matplotlib inline

In [2]:
#Author: Proloy Das <pdas6@mgh.harvard.edu>
#
# License: MIT

In [3]:
import mne
import numpy as np
from mne.datasets import sample
from purdonlabmeeg._spatial_map_utils._custom_solver import apply_solver

Example showing source localization within a Hierarchial Bayes Framework. 

In [4]:

data_path = sample.data_path()
subject = 'sample'
meg_path = data_path / 'MEG' / 'sample'
raw_fname = meg_path / 'sample_audvis_filt-0-40_raw.fif'
event_fname = meg_path / 'sample_audvis_filt-0-40_raw-eve.fif'
fwd_fname = meg_path / 'sample_audvis-meg-eeg-oct-6-fwd.fif'
trans = meg_path / 'sample_audvis_raw-trans.fif'
ave_fname = meg_path / 'sample_audvis-ave.fif'
cov_fname = meg_path / 'sample_audvis-cov.fif'
subjects_dir = data_path / 'subjects'
bem_sol_fname = subjects_dir / 'sample' / 'bem' / 'sample-5120-5120-5120-bem-sol.fif'
condition = 'Left Auditory'

In [5]:
# Read noise covariance matrix
noise_cov = mne.read_cov(cov_fname)


    366 x 366 full covariance (kind = 1) found.
    Read a total of 4 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
        Average EEG reference (1 x 60) active


In [6]:
# Handling raw files 
raw = mne.io.read_raw_fif(raw_fname)
events = mne.read_events(event_fname)
raw.info['bads'] += ['MEG 2443']
tmin = -0.2
tmax = 0.3  # Use a lower tmax to reduce multiple comparisons
picks = mne.pick_types(raw.info, meg=True, eeg=True, eog=True, exclude='bads')
event_id = [1,]  # L auditory
reject = dict(grad=1000e-13, mag=4000e-15, eog=150e-6)
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, picks=picks,
                    baseline=(None, 0), reject=reject, preload=True)
evoked = epochs.pick_types(eeg=True, meg=True)
evoked = epochs.average(picks=['meg', 'eeg', 'eog'])

Opening raw data file /homes/1/pd640/Workspace/MNE-sample-data/MEG/sample/sample_audvis_filt-0-40_raw.fif...
    Read a total of 4 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
        Average EEG reference (1 x 60)  idle
    Range : 6450 ... 48149 =     42.956 ...   320.665 secs
Ready.
Not setting metadata
72 matching events found
Setting baseline interval to [-0.19979521315838786, 0.0] sec
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 4)
4 projection items activated
Loading data for 72 events and 76 original time points ...
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on MAG : ['MEG 1711']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch bas

In [7]:
# Handling forward solution
forward = mne.read_forward_solution(fwd_fname)
src = mne.setup_source_space('fsaverage', spacing='ico4', subjects_dir=subjects_dir,
                                n_jobs=-1, add_dist=False)
src = mne.morph_source_spaces(src, 'sample', subjects_dir=subjects_dir)
bem = mne.read_bem_solution(bem_sol_fname)
fwd = mne.make_forward_solution(raw.info, trans, src=src, bem=bem,
                                meg=True, eeg=True, n_jobs=-1)

Reading forward solution from /homes/1/pd640/Workspace/MNE-sample-data/MEG/sample/sample_audvis-meg-eeg-oct-6-fwd.fif...
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523) not available
    Read MEG forward solution (7498 sources, 306 channels, free orientations)
    Desired named matrix (kind = 3523) not available
    Read EEG forward solution (7498 sources, 60 channels, free orientations)
    MEG and EEG forward solutions combined
    Source spaces transformed to the forward solution coordinate frame
Setting up the source space with the following parameters:

SUBJECTS_DIR = /homes/1/pd640/Workspace/MNE-sample-data/subjects
Subject      = fsaverage
Surface      = white
Icosahedron subdivision grad

[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   8 out of  32 | elapsed:   13.1s remaining:   39.2s
[Parallel(n_jobs=32)]: Done  15 out of  32 | elapsed:   13.5s remaining:   15.3s
[Parallel(n_jobs=32)]: Done  22 out of  32 | elapsed:   13.7s remaining:    6.2s
[Parallel(n_jobs=32)]: Done  29 out of  32 | elapsed:   13.9s remaining:    1.4s


    Found    0/2016 points outside using solid angles
    Total 2559/2562 points inside the surface
Interior check completed in 14043.6 ms
    3 source space points omitted because they are outside the inner skull surface.
Checking surface interior status for 2562 points...
    Found  532/2562 points inside  an interior sphere of radius   43.6 mm
    Found    0/2562 points outside an exterior sphere of radius   91.8 mm
    Found    2/2030 points outside using surface Qhull
    Found    0/2028 points outside using solid angles
    Total 2560/2562 points inside the surface
Interior check completed in 112.8 ms
    2 source space points omitted because they are outside the inner skull surface.



[Parallel(n_jobs=32)]: Done  32 out of  32 | elapsed:   14.0s finished
[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   8 out of  32 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=32)]: Done  15 out of  32 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=32)]: Done  22 out of  32 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=32)]: Done  29 out of  32 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=32)]: Done  32 out of  32 | elapsed:    0.1s finished


Checking surface interior status for 306 points...
    Found   0/306 points inside  an interior sphere of radius   79.1 mm
    Found   0/306 points outside an exterior sphere of radius  161.3 mm
    Found 306/306 points outside using surface Qhull
    Found   0/  0 points outside using solid angles
    Total 0/306 points inside the surface
Interior check completed in 57.2 ms
Setting up compensation data...
    No compensation set. Nothing more to do.

Composing the field computation matrix...


...

Setting up for EEG...
Computing MEG at 5119 source locations (free orientations)...


[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   8 out of  32 | elapsed:    0.7s remaining:    2.0s
[Parallel(n_jobs=32)]: Done  15 out of  32 | elapsed:    0.9s remaining:    1.0s
[Parallel(n_jobs=32)]: Done  22 out of  32 | elapsed:    1.1s remaining:    0.5s
[Parallel(n_jobs=32)]: Done  29 out of  32 | elapsed:    1.4s remaining:    0.1s
[Parallel(n_jobs=32)]: Done  32 out of  32 | elapsed:    1.5s finished


Computing EEG at 5119 source locations (free orientations)...


[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   8 out of  32 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=32)]: Done  15 out of  32 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=32)]: Done  22 out of  32 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=32)]: Done  29 out of  32 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=32)]: Done  32 out of  32 | elapsed:    0.1s finished
[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   8 out of  32 | elapsed:    0.2s remaining:    0.6s
[Parallel(n_jobs=32)]: Done  15 out of  32 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=32)]: Done  22 out of  32 | elapsed:    0.3s remaining:    0.1s
[Parallel(n_jobs=32)]: Done  29 out of  32 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=32)]: Done  32 out of  32 | elapsed:    0.3s finished



Finished.


In [8]:
stc, labels, nu = apply_solver(epochs, fwd, noise_cov, loose=0., depth=0.5,
                    extent=[9,], local_smoothness=0.9999, subject=subject, subjects_dir=subjects_dir,
                    average_first=True)


Converting forward solution to fixed orientation
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]
info["bads"] and noise_cov["bads"] do not match, excluding bad channels from both
Computing inverse operator with 364 channels.
    364 out of 366 channels remain after picking
Selected 364 channels
Creating the depth weighting matrix...
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 4)
Computing rank from covariance with rank=None
    Using tolerance 3.3e-13 (2.2e-16 eps * 305 dim * 4.8  max singular value)
    Estimated rank (mag + grad): 302
    MEG: rank 302 computed from 305 data channels with 3 projectors
    Using tolerance 4.7e-14 (2.2e-16 eps * 59 dim * 3.6  max singular value)
    Estimated rank (eeg): 58
    EEG: rank 58 computed from 59 data channels with 1 p

/autofs/cluster/purdonlab/users/proloy/purdonlabmeeg/purdonlabmeeg/_spatial_map_utils/_parcel_utils.py:503: RuntimeWarning: 0.1% of original source space vertices have been omitted, tri-based adjacency will have holes.
Consider using distance-based adjacency or morphing data to all source space vertices.
  adjacency = spatial_src_adjacency(fwd['src'])


isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sourc

/autofs/cluster/purdonlab/users/proloy/purdonlabmeeg/purdonlabmeeg/_spatial_map_utils/_custom_solver.py:132: RuntimeWarning: 0.1% of original source space vertices have been omitted, tri-based adjacency will have holes.
Consider using distance-based adjacency or morphing data to all source space vertices.
  adjacency = mne.spatial_src_adjacency(forward['src'])
100%|██████████| 500/500 [00:48<00:00, 10.22it/s]


[8.13647126e+00 1.91014314e+00 8.69244426e-02 1.98108633e-01
 1.91457957e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 3.17991969e-02 0.00000000e+00
 6.39173414e-01 2.47183838e+00 5.42668854e-02 1.67189825e-01
 7.48663025e-01 0.00000000e+00 1.22819946e-01 0.00000000e+00
 0.00000000e+00 3.29191173e-02 0.00000000e+00 2.50931395e-02
 0.00000000e+00 5.21060027e-01 1.23556612e-01 0.00000000e+00
 0.00000000e+00 8.67362194e-07 0.00000000e+00 1.05714518e-06
 2.12369509e-01 7.50083014e-02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 3.95802005e-02 0.00000000e+00 9.45268939e-01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 4.77135360e-02
 2.93226765e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 7.81832833e-07 4.79999634e-05
 0.00000000e+00 1.04226661e+00 8.28064186e-03 1.90234828e-01
 6.16263169e-01 0.00000000e+00 4.78252752e-02 0.00000000e+00
 4.80572024e-02 0.00000000e+00 1.64033067e-02 0.00000000e+00
 0.00000000e+00 4.017706

In [9]:
brain = stc.plot(surface='white', hemi='both', subjects_dir=subjects_dir, alpha=0.8)

Using pyvistaqt 3d backend.

Using control points [1.03556738e-09 1.57193410e-09 9.64436915e-09]


qt.qpa.xcb: QXcbConnection: XCB error: 148 (Unknown), sequence: 192, resource id: 0, major code: 140 (Unknown), minor code: 20


In [10]:
stc, labels, nu = apply_solver(epochs.crop(0.04, 0.21), fwd, noise_cov, loose=0., depth=0.5,
                    extent=[9,], local_smoothness=0.9999, subject=subject, subjects_dir = subjects_dir,
                    average_first=False)


Converting forward solution to fixed orientation
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]
info["bads"] and noise_cov["bads"] do not match, excluding bad channels from both
Computing inverse operator with 364 channels.
    364 out of 366 channels remain after picking
Selected 364 channels
Creating the depth weighting matrix...
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 4)
Computing rank from covariance with rank=None
    Using tolerance 3.3e-13 (2.2e-16 eps * 305 dim * 4.8  max singular value)
    Estimated rank (mag + grad): 302
    MEG: rank 302 computed from 305 data channels with 3 projectors
    Using tolerance 4.7e-14 (2.2e-16 eps * 59 dim * 3.6  max singular value)
    Estimated rank (eeg): 58
    EEG: rank 58 computed from 59 data channels with 1 p

/autofs/cluster/purdonlab/users/proloy/purdonlabmeeg/purdonlabmeeg/_spatial_map_utils/_parcel_utils.py:503: RuntimeWarning: 0.1% of original source space vertices have been omitted, tri-based adjacency will have holes.
Consider using distance-based adjacency or morphing data to all source space vertices.
  adjacency = spatial_src_adjacency(fwd['src'])


isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sources
isolated sourc

/autofs/cluster/purdonlab/users/proloy/purdonlabmeeg/purdonlabmeeg/_spatial_map_utils/_custom_solver.py:132: RuntimeWarning: 0.1% of original source space vertices have been omitted, tri-based adjacency will have holes.
Consider using distance-based adjacency or morphing data to all source space vertices.
  adjacency = mne.spatial_src_adjacency(forward['src'])
100%|██████████| 500/500 [00:52<00:00,  9.58it/s]


[5.79073252e+02 1.70633577e+02 5.03156722e+00 4.28776777e-03
 7.18319253e+00 0.00000000e+00 9.12964913e-03 3.76225226e-01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.82637594e+00 6.29173542e+00 3.51105448e-01 1.31370132e+00
 1.95398261e+00 0.00000000e+00 1.12851301e+00 2.06952059e-01
 0.00000000e+00 5.33297696e-03 0.00000000e+00 1.49283823e+00
 0.00000000e+00 1.16672628e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 3.04924012e-05 0.00000000e+00 1.03018700e+00
 2.07586981e-04 0.00000000e+00 0.00000000e+00 3.75702626e-01
 7.74538345e-02 1.27643457e-01 3.72203684e-03 1.72755074e+01
 0.00000000e+00 0.00000000e+00 1.04733182e+00 4.44350435e-02
 0.00000000e+00 4.92625640e-04 2.04590542e-02 0.00000000e+00
 0.00000000e+00 6.54203079e-01 3.49656993e+00 1.53872340e-02
 4.63342156e-01 1.65773027e+00 0.00000000e+00 2.86767996e-02
 5.06470956e-01 0.00000000e+00 1.11557339e+00 0.00000000e+00
 4.39086110e-01 3.56683252e-01 2.98454258e-01 0.00000000e+00
 0.00000000e+00 7.873939

In [11]:
brain = stc.plot(surface='white', hemi='both', subjects_dir=subjects_dir, alpha=0.8)

Using control points [8.22367241e-10 1.19591406e-09 4.99033595e-09]
